2019 Census Data for Selected Variables - Baltimore City

In [ ]:
#pip install us

In [ ]:
# pip install censusgeocode

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata as c
import censusgeocode as cg
#import gmaps

# Census & gmaps API Keys
from config import api_key
c = Census(api_key, year=2019)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [13]:
# Run Census Search to retrieve data on all states
# ex. "B23025_005E" is unemployment count
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B08136_003E",
                          "B23025_004E",
                          "B23025_005E",                                    
                          "B15003_017E"), 
                         state_fips = states.MD.fips,
                         county_fips = "510",
                         tract = "*")

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B08136_003E,B23025_004E,B15003_017E,state,county,tract
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,NaN,885.0,450.0,24,510,260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,NaN,1500.0,72.0,24,510,230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,NaN,2366.0,1231.0,24,510,260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,NaN,286.0,229.0,24,510,260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,NaN,1233.0,394.0,24,510,260800


In [4]:
# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Median Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B08136_003E": "Time spent commuting (minutes)",
                                      "B23025_004E": "Number of Employed, age 16+",
                                      "B23025_005E": "Unemployment Count",
                                      "B15003_017E": "Number of persons age 25+ graduated high school",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Median Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate", 
                       "Time spent commuting (minutes)","Number of Employed, age 16+", "Unemployment Count",
                      "Number of persons age 25+ graduated high school"]]

census_pd.head()

,State,Name,Population,Median Age,Median Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Time spent commuting (minutes),"Number of Employed, age 16+",Unemployment Count,Number of persons age 25+ graduated high school
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,27.9,31625.0,21744.0,436.0,24.716553,1.927438,NaN,885.0,34.0,450.0
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,30.7,107391.0,63973.0,151.0,7.618567,0.857719,NaN,1500.0,17.0,72.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,41.3,62586.0,25903.0,469.0,9.821990,3.162304,NaN,2366.0,151.0,1231.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,37.0,17000.0,19562.0,377.0,38.906089,2.063983,NaN,286.0,20.0,229.0
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,36.1,40795.0,26895.0,481.0,20.626072,1.629503,NaN,1233.0,38.0,394.0


In [5]:
census_pd["Poverty Rate"] = census_pd["Poverty Rate"].astype(float).round(1)
census_pd.head()

,State,Name,Population,Median Age,Median Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Time spent commuting (minutes),"Number of Employed, age 16+",Unemployment Count,Number of persons age 25+ graduated high school
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,27.9,31625.0,21744.0,436.0,24.7,1.927438,NaN,885.0,34.0,450.0
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,30.7,107391.0,63973.0,151.0,7.6,0.857719,NaN,1500.0,17.0,72.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,41.3,62586.0,25903.0,469.0,9.8,3.162304,NaN,2366.0,151.0,1231.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,37.0,17000.0,19562.0,377.0,38.9,2.063983,NaN,286.0,20.0,229.0
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,36.1,40795.0,26895.0,481.0,20.6,1.629503,NaN,1233.0,38.0,394.0


In [6]:
census_pd["Unemployment Rate"] = census_pd["Unemployment Rate"].astype(float).round(1)
census_pd.head()

,State,Name,Population,Median Age,Median Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Time spent commuting (minutes),"Number of Employed, age 16+",Unemployment Count,Number of persons age 25+ graduated high school
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,27.9,31625.0,21744.0,436.0,24.7,1.9,NaN,885.0,34.0,450.0
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,30.7,107391.0,63973.0,151.0,7.6,0.9,NaN,1500.0,17.0,72.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,41.3,62586.0,25903.0,469.0,9.8,3.2,NaN,2366.0,151.0,1231.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,37.0,17000.0,19562.0,377.0,38.9,2.1,NaN,286.0,20.0,229.0
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,36.1,40795.0,26895.0,481.0,20.6,1.6,NaN,1233.0,38.0,394.0


In [7]:
census_pd.count()

State                                              200
Name                                               200
Population                                         200
Median Age                                         200
Median Household Income                            200
Per Capita Income                                  200
Poverty Count                                      200
Poverty Rate                                       199
Unemployment Rate                                  199
Time spent commuting (minutes)                      16
Number of Employed, age 16+                        200
Unemployment Count                                 200
Number of persons age 25+ graduated high school    200
dtype: int64

In [8]:

census_pd[['Census_Tract', "County", "State Name"]]= census_pd['Name'].str.split(",", n=3, expand=True)
census_pd.head()

,State,Name,Population,Median Age,Median Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Time spent commuting (minutes),"Number of Employed, age 16+",Unemployment Count,Number of persons age 25+ graduated high school,Census_Tract,County,State Name
0,24,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,27.9,31625.0,21744.0,436.0,24.7,1.9,NaN,885.0,34.0,450.0,Census Tract 2604.03,Baltimore city,Maryland
1,24,"Census Tract 2302, Baltimore city, Maryland",1982.0,30.7,107391.0,63973.0,151.0,7.6,0.9,NaN,1500.0,17.0,72.0,Census Tract 2302,Baltimore city,Maryland
2,24,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,41.3,62586.0,25903.0,469.0,9.8,3.2,NaN,2366.0,151.0,1231.0,Census Tract 2601.02,Baltimore city,Maryland
3,24,"Census Tract 2603.03, Baltimore city, Maryland",969.0,37.0,17000.0,19562.0,377.0,38.9,2.1,NaN,286.0,20.0,229.0,Census Tract 2603.03,Baltimore city,Maryland
4,24,"Census Tract 2608, Baltimore city, Maryland",2332.0,36.1,40795.0,26895.0,481.0,20.6,1.6,NaN,1233.0,38.0,394.0,Census Tract 2608,Baltimore city,Maryland


In [9]:
census_pd = census_pd.drop(["State", "Time spent commuting (minutes)"], axis=1)
census_pd.head()

,Name,Population,Median Age,Median Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,"Number of Employed, age 16+",Unemployment Count,Number of persons age 25+ graduated high school,Census_Tract,County,State Name
0,"Census Tract 2604.03, Baltimore city, Maryland",1764.0,27.9,31625.0,21744.0,436.0,24.7,1.9,885.0,34.0,450.0,Census Tract 2604.03,Baltimore city,Maryland
1,"Census Tract 2302, Baltimore city, Maryland",1982.0,30.7,107391.0,63973.0,151.0,7.6,0.9,1500.0,17.0,72.0,Census Tract 2302,Baltimore city,Maryland
2,"Census Tract 2601.02, Baltimore city, Maryland",4775.0,41.3,62586.0,25903.0,469.0,9.8,3.2,2366.0,151.0,1231.0,Census Tract 2601.02,Baltimore city,Maryland
3,"Census Tract 2603.03, Baltimore city, Maryland",969.0,37.0,17000.0,19562.0,377.0,38.9,2.1,286.0,20.0,229.0,Census Tract 2603.03,Baltimore city,Maryland
4,"Census Tract 2608, Baltimore city, Maryland",2332.0,36.1,40795.0,26895.0,481.0,20.6,1.6,1233.0,38.0,394.0,Census Tract 2608,Baltimore city,Maryland


In [10]:
census_pd.columns

Index(['Name', 'Population', 'Median Age', 'Median Household Income',
       'Per Capita Income', 'Poverty Count', 'Poverty Rate',
       'Unemployment Rate', 'Number of Employed, age 16+',
       'Unemployment Count', 'Number of persons age 25+ graduated high school',
       'Census_Tract', 'County', 'State Name'],
      dtype='object')

In [ ]:
# calculate daytime population; from: https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# Two methods for calculating commuter-adjusted population estimates:

#Method 1: Total resident population + (total workers working in area - workers who lived and worked in same area) - (total workers living in area - workers who lived and worked in same area) OR;
#Method 2: Total resident population + Total workers working in area - Total workers living in area.




In [ ]:
# Reorganize the columns using double brackets
census_pd = census_pd[["State Name","County","Census_Tract","Population",'Median Age', 'Median Household Income', 'Per Capita Income',
       'Poverty Count', 'Poverty Rate', 'Unemployment Rate',
       'Time spent commuting (minutes)', 'Number of Employed, age 16+',
       'Unemployment Count', 'Number of persons age 25+ graduated high school']]
census_pd

In [ ]:
census_pd["Census_Tract"] = census_pd['Census_Tract'].str.replace('Census Tract', "") 
census_pd

In [ ]:
# Calculations can also be performed on Series and added into DataFrames as new columns
#thousands_of_dollars = data_file_df["Amount"]/1000
#data_file_df["Thousands of Dollars"] = thousands_of_dollars

#data_file_df.head()

In [ ]:
# Converting the membership days into weeks and then adding a column to the DataFrame
#weeks = training_df["Membership (Days)"]/7
#training_df["Membership (Weeks)"] = weeks

#training_df.head()

In [ ]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_pd["Census_Tract"].unique())
tract_count

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
census_pd.to_csv("Comm_Corridor_Stats_Draft1.csv", index = False, header=True)